In [226]:
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd

tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf=True)
tfidf_transformer.fit(word_count_vector)


#TF-IDF 계산 및 키워드 추출
#read test docs into a dataframe and concatenate title and body
paramnames = ['rank','id','title','content'] 

df_idf = pd.read_csv("D:\\Project\\WebCrowling\\tt00000.tsv",sep='\t', error_bad_lines=False, names=paramnames)
df_idf['text'] = df_idf['title'] + df_idf['content']
df_idf['text'] = df_test['text'].apply(lambda x:pre_process(x))

#get test docs into a list
docs_test = df_test['text']
print(df_idf.dtypes)

rank        int64
id         object
title      object
content    object
text       object
dtype: object


In [227]:
import re
def pre_process(text):
    text=text.lower()
    
    #remove tag
    text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    #remove special characters and digits
    text = re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf['text'] = df_idf['title'] + df_idf['content']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

df_idf['text']

0       tied for the best movie i have ever seenwhy do...
1       a classic piece of unforgettable film making i...
2       prepare to be movedi have never seen such an a...
3       this is how movies should be madethis movie is...
4       shawshank redeems hollywoodcan hollywood usual...
5       don t rent shawshank i m trying to save you mo...
6       some birds aren t meant to be caged the shawsh...
7       eternal hopei ve lost count of the number of t...
8       stephen king s best adapted moviemisery and st...
9       the shawshank redemptionone of the finest film...
10      two movies in onethe reason i became a member ...
11      simply known as the movie whenever i talk abou...
12      simply amazing the best film of the s the shaw...
13      its an excellent piece of arti watch this movi...
14      the absolute greatest movie of all time my lif...
15      the best story ever told on filmi believe that...
16      oh my godmotherthree words oh my god how am i ...
17      i make

In [228]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words(stop_file_path):
    with open(stop_file_path, 'r', encoding = "utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)
    
#load a set of stopwords
stopwords = get_stop_words("stopwords.txt")

#get the text column
docs = df_idf['text'].tolist()

#create a vocabulary of words,
#ignore word hat appear in 85% of documents,
#eliminate stop words
cv = CountVectorizer(max_df = 0.85,stop_words = stopwords)
word_count_vector=cv.fit_transform(docs)



In [229]:
cv=CountVectorizer(max_df=0.85, stop_words = stopwords, max_features = 10000)
word_count_vector=cv.fit_transform(docs)

list(cv.vocabulary_.keys())[:10]


['tied',
 'best',
 'movie',
 'write',
 'th',
 'comment',
 'shawshank',
 'redemption',
 'am',
 'sure']

In [195]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf=True)
tfidf_transformer.fit(word_count_vector)


#TF-IDF 계산 및 키워드 추출
#read test docs into a dataframe and concatenate title and body
paramnames = ['rank','id','title','content'] 

df_idf = pd.read_csv("D:\\Project\\WebCrowling\\tt00000.tsv",sep='\t', error_bad_lines=False, names=paramnames)
df_test['text'] = df_idf['title'] + df_idf['content']
df_test['text'] = df_test['text'].apply(lambda x:pre_process(x))

#get test docs into a list
docs_test = df_test['text']
print(docs_test)

0      tied for the best movie i have ever seenwhy do...
1      a classic piece of unforgettable film making i...
2      prepare to be movedi have never seen such an a...
3      this is how movies should be madethis movie is...
4      shawshank redeems hollywoodcan hollywood usual...
5      don t rent shawshank i m trying to save you mo...
6      some birds aren t meant to be caged the shawsh...
7      eternal hopei ve lost count of the number of t...
8      stephen king s best adapted moviemisery and st...
9      the shawshank redemptionone of the finest film...
10     two movies in onethe reason i became a member ...
11     simply known as the movie whenever i talk abou...
12     simply amazing the best film of the s the shaw...
13     its an excellent piece of arti watch this movi...
14     the absolute greatest movie of all time my lif...
15     the best story ever told on filmi believe that...
16     oh my godmotherthree words oh my god how am i ...
17     i make the questions her

In [169]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1],x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """gety the feature names and tf-idf score of top n items"""
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    
    #word index and corresponding tf-idf score
    for idx, score in sorted_items:
        #keep track of feature name and its corresponding score
        score_vals.append(round(score,3))
        feature_vals.append(feature_names[idx])
        
    #create a tuples of feature,score
    #results -= zip(feature_vals,score_vals)
    results = {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
        
    return results


In [260]:
#you only needs to do this once, this is a mapping of index to
feature_names = cv.get_feature_names()


#for i in range(len(df_idf['text'])):

#get the document that we want to extract keywords from

#generate tf-idf for the given document
tf_idf_vector = tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items = sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords = extract_topn_from_vector(feature_names, sorted_items, 10)

#now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")

Temp = []
for i in range(1,5000): # 5000까지 반복
    doc = df_idf['content'][i] #content의 i에 해당하는걸 doc에
    for k in keywords: #k 내 키워드를 검사
        Temp.append()[k]
            
print(a)


=====Doc=====
In its Oscar year, Shawshank Redemption (written and directed by Frank Darabont, after the novella Rita Hayworth and the Shawshank Redemption, by Stephen King) was nominated for seven Academy Awards, and walked away with zero. Best Picture went to Forrest Gump, while Shawshank and Pulp Fiction were "just happy to be nominated." Of course hindsight is 20/20, but while history looks back on Gump as a good film, Pulp and Redemption are remembered as some of the all-time best. Pulp, however, was a success from the word "go," making a huge splash at Cannes and making its writer-director an American master after only two films. For Andy Dufresne and Co., success didn't come easy. Fortunately, failure wasn't a life sentence.After opening on 33 screens with take of $727,327, the $25M film fell fast from theatres and finished with a mere $28.3M. The reasons for failure are many. Firstly, the title is a clunker. While iconic to fans today, in 1994, people knew not and cared not wh

TypeError: append() takes exactly one argument (0 given)